In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, PegasusForConditionalGeneration, PegasusTokenizer
import torch
import pandas as pd
import torch
import time
from rouge import Rouge

In [ ]:
rouge = Rouge()

In [ ]:
ARTICLE = """ Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. "I'll definitely have some sort of party," he said in an interview. "Hopefully none of you will be reading about it." Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. "People are always looking to say 'kid star goes off the rails,'" he told reporters last month. "But I try very hard not to go that way because it would be too easy for them." His latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called "My Boy Jack," about author Rudyard Kipling and his son, due for release later this year. He will also appear in "December Boys," an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's "Equus." Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: "I just think I'm going to be more sort of fair game," he told Reuters.
"""

In [ ]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)
pipe = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)

In [ ]:
# Use a pipeline as a high-level helper
start_time = time.time()
summary = summarizer(ARTICLE, max_length=300, min_length=200, do_sample=False)[0].get('summary_text')
end_time = time.time()
print(end_time - start_time)
print(summary)

In [ ]:
rouge = Rouge()

In [ ]:

sum = """
Harry Potter star Daniel Radcliffe gains access to a reported £20 million fortune .<n>The young actor says he has no plans to fritter his cash away .<n>Radcliffe's earnings from the first five Potter films have been held in a trust fund .<n>His latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix" is breaking records .<n>The Londoner has filmed a TV movie called "My Boy Jack," about author Rudyard Kipling .<n>He will also appear in "December Boys," an Australian film about four boys who escape an orphanage .<n>Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's "Equus"<n>He will reprise the role in the last two Potter films and will also appear in "My Boy Jack," "December Boys" and "Harry Potter and the Order of the Phoenix," later this year .<n>The Londoner has filmed a TV movie called "My Boy Jack," about Rudyard Kipling and his son .
"""


In [ ]:
len(ARTICLE.split(' '))

In [ ]:
len(reference.split())

In [ ]:
start_time = time.time()
summary2 = pipe(ARTICLE, max_length=400, min_length=300, do_sample=False)[0].get('summary_text')
end_time = time.time()
print(end_time - start_time)

In [ ]:
print(summary2)

In [ ]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail", tokenizer="facebook/bart-large-cnn", device=0)

In [ ]:
# Load model directly
src_text = [
    """ PG&E stated it scheduled the blackouts in response to forecasts for high winds amid dry conditions. The aim is to reduce the risk of wildfires. Nearly 800 thousand customers were scheduled to be affected by the shutoffs which were expected to last through at least midday tomorrow."""
]
model_name = "google/pegasus-cnn_dailymail"
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)
batch = tokenizer(src_text, truncation=True, padding="longest", return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
assert (
        tgt_text[0]
        == "California's largest electricity provider has turned off power to hundreds of thousands of customers."
)

In [ ]:
if torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model_path = '../fine-tuned-pegasus/model'
tokenizer_path = '../fine-tuned-pegasus/tokenizer'
# 加载保存的模型

model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)

# 加载保存的分词器
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
rouge = Rouge()

In [ ]:
inputs = tokenizer(
    ARTICLE, return_tensors="pt", truncation=True , padding="max_length", max_length=512
).to(device)

summary_ids = model.generate(
    inputs['input_ids'],
    max_length=300,
    min_length=100,
    length_penalty=2.0,
    num_beams=8,
    early_stopping=False,
    no_repeat_ngram_size=2
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
tokenizer.decode(inputs[0], skip_special_tokens=True)

In [ ]:
summary

In [ ]:
reference = """
Daniel Radcliffe, the actor famous for playing Harry Potter, turns 18 on Monday and will gain access to a reported £20 million ($41.1 million) fortune. Despite the significant wealth, Radcliffe insists he has no plans to spend extravagantly on fast cars, alcohol, or celebrity parties. He prefers to spend his money on books, CDs, and DVDs. At 18, he will also be able to legally gamble, buy alcohol, and watch restricted films like "Hostel: Part II." Radcliffe's earnings from the first five Harry Potter films have been held in a trust fund, and he has maintained a grounded approach to his newfound wealth and fame. His latest film, "Harry Potter and the Order of the Phoenix," is breaking box office records, and he will continue the role in the final two films. Beyond the Potter franchise, Radcliffe has taken on diverse roles, including a TV movie about Rudyard Kipling and his son, and an Australian film called "December Boys." He also made his stage debut in "Equus." Radcliffe anticipates increased media scrutiny now that he is legally an adult but remains committed to staying grounded.
"""
rouge.get_scores(summary,reference)

In [ ]:
input_ids = inputs['input_ids']
input_ids.shape[1]

In [ ]:
len(reference.split())

In [ ]:
len(ARTICLE.split())

In [ ]:
import torch
import tensorflow as tf
from datasets import load_dataset, load_from_disk
import accelerate
from transformers import (
    PegasusTokenizer,
    PegasusForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
if torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print(device)

In [ ]:
import pandas as pd
import json

In [ ]:
import json

# Step 1: Load the JSON file
with open('/Users/tangyunpei/Documents/FDU/Fall2024/CSCI6806V4CapStone/results/checkpoint-129201/trainer_state.json', 'r') as f:
    trainer_state = json.load(f)

# Step 2: Extract the `log_history` part
log_history = trainer_state.get("log_history", [])

In [ ]:
df_log_history = pd.DataFrame(log_history)

# Add an index starting from 1
df_log_history.index = df_log_history.index + 1


In [ ]:
df_log_history


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax1 = plt.subplots(figsize=(12, 8))

# Plot Loss on the first y-axis (ax1)
color = 'tab:blue'
ax1.set_xlabel('Step')
ax1.set_ylabel('Loss', color=color)
if 'loss' in df_log_history.columns:
    ax1.plot(df_log_history['step'], df_log_history['loss'], label='Loss', color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Step 6: Create a second y-axis (ax2) for Grad Norm
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:green'
ax2.set_ylabel('Grad Norm', color=color)
if 'grad_norm' in df_log_history.columns:
    ax2.plot(df_log_history['step'], df_log_history['grad_norm'], label='Grad Norm', color=color)
ax2.tick_params(axis='y', labelcolor=color)

# Step 7: Create a third y-axis (ax3) for Learning Rate
ax3 = ax1.twinx()  # instantiate a third axes that shares the same x-axis
ax3.spines['right'].set_position(('outward', 60))  # offset the third y-axis
color = 'tab:red'
ax3.set_ylabel('Learning Rate', color=color)
if 'learning_rate' in df_log_history.columns:
    ax3.plot(df_log_history['step'], df_log_history['learning_rate'], label='Learning Rate', color=color)
ax3.tick_params(axis='y', labelcolor=color)

# Step 8: Add title and show the plot
plt.title('Loss, Grad Norm, and Learning Rate over Steps')
plt.show()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir=./results/runs

In [ ]:
ARTICLE = """
Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. "I'll definitely have some sort of party," he said in an interview. "Hopefully none of you will be reading about it." Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. "People are always looking to say 'kid star goes off the rails,'" he told reporters last month. "But I try very hard not to go that way because it would be too easy for them." His latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called "My Boy Jack," about author Rudyard Kipling and his son, due for release later this year. He will also appear in "December Boys," an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's "Equus." Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: "I just think I'm going to be more sort of fair game," he told Reuters.
"""

In [ ]:
# 模型和分词器加载
model = AutoModelForSeq2SeqLM.from_pretrained('/Users/tangyunpei/Documents/FDU/Fall2024/CSCI6806V4CapStone/CSCI6806Capstone/model').to(device)
tokenizer = AutoTokenizer.from_pretrained('/Users/tangyunpei/Documents/FDU/Fall2024/CSCI6806V4CapStone/CSCI6806Capstone/model')

In [ ]:
# 定义最大长度
max_token_length = 256  # 模型能够处理的最大长度

# 对文章进行分段
inputs = tokenizer(ARTICLE, return_tensors="pt", padding=False, truncation=False, max_length=max_token_length)
input_ids = inputs['input_ids'][0].tolist()  # 转换为列表，方便切片操作

# 分割文章：每次取 max_token_length 长度的部分
chunks = [input_ids[i:i + max_token_length] for i in range(0, len(input_ids), max_token_length)]

# 定义一个函数来生成每段的摘要
def generate_summary(input_ids_chunk):
    inputs_chunk = {"input_ids": torch.tensor([input_ids_chunk]).to(device)}  # 转换为 PyTorch 张量
    summary_ids = model.generate(
        inputs_chunk['input_ids'],
        max_length=300,  # 根据需要设置每段摘要的最大长度
        min_length=50,   # 根据需要设置每段摘要的最小长度
        length_penalty=1.5,
        num_beams=10,
        early_stopping=True,
    )
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 对每一部分生成摘要
summaries = [generate_summary(chunk) for chunk in chunks]

# 最终的汇总摘要
final_summary = " ".join(summaries)
print(final_summary)

In [ ]:
final_summary = " ".join(summaries[:])

In [ ]:
final_summary = """
Daniel Radcliffe, the 18-year-old star of "Harry Potter and the Order of the Phoenix," is set to turn 18 on Monday, with a reported £20 million ($41.1 million) fortune. Despite the wealth, Radcliffe insists he will not spend it on extravagant items like fast cars, drink, or celebrity parties. He prefers books, CDs, and DVDs costing about 10 pounds. Radcliffe will be able to gamble in a casino, buy a drink in a pub, or watch the horror film "Hostel: Part II," which currently ranks sixth on the UK box office chart. His agent and publicist have not commented on his birthday plans Daniel Radcliffe, the actor known for his role as Harry Potter in the "Harry Potter" series, has revealed that his earnings from the first five films have been held in a trust fund, which he has not been able to access. Despite his growing fame and wealth, Radcliffe remains grounded, stating, "People are always looking to say 'kid star goes off the rails.' But I try very hard not to go that way because it would be too easy for them." His latest film, "Harry Potter and the Order of the Phoenix," is breaking records in both the UK and the U.S., and he will reprise **Summary:** In an interview with Reuters, the author emphasized the importance of fair play, stating that the game is more akin to a fair game. The article also mentioned that the author plans to publish a book on the concept of fair play in the coming months.
"""

In [ ]:
summary


In [ ]:
ARTICLE = """
Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don't think I'll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office chart. Details of how he'll mark his landmark birthday are under wraps. His agent and publicist had no comment on his plans. "I'll definitely have some sort of party," he said in an interview. "Hopefully none of you will be reading about it." Radcliffe's earnings from the first five Potter films have been held in a trust fund which he has not been able to touch. Despite his growing fame and riches, the actor says he is keeping his feet firmly on the ground. "People are always looking to say 'kid star goes off the rails,'" he told reporters last month. "But I try very hard not to go that way because it would be too easy for them." His latest outing as the boy wizard in "Harry Potter and the Order of the Phoenix" is breaking records on both sides of the Atlantic and he will reprise the role in the last two films. Watch I-Reporter give her review of Potter's latest » . There is life beyond Potter, however. The Londoner has filmed a TV movie called "My Boy Jack," about author Rudyard Kipling and his son, due for release later this year. He will also appear in "December Boys," an Australian film about four boys who escape an orphanage. Earlier this year, he made his stage debut playing a tortured teenager in Peter Shaffer's "Equus." Meanwhile, he is braced for even closer media scrutiny now that he's legally an adult: "I just think I'm going to be more sort of fair game," he told Reuters.	
"""
reference = """
Daniel Radcliffe, the actor famous for playing Harry Potter, turns 18 on Monday and will gain access to a reported £20 million ($41.1 million) fortune. Despite the significant wealth, Radcliffe insists he has no plans to spend extravagantly on fast cars, alcohol, or celebrity parties. He prefers to spend his money on books, CDs, and DVDs. At 18, he will also be able to legally gamble, buy alcohol, and watch restricted films like "Hostel: Part II." Radcliffe's earnings from the first five Harry Potter films have been held in a trust fund, and he has maintained a grounded approach to his newfound wealth and fame. His latest film, "Harry Potter and the Order of the Phoenix," is breaking box office records, and he will continue the role in the final two films. Beyond the Potter franchise, Radcliffe has taken on diverse roles, including a TV movie about Rudyard Kipling and his son, and an Australian film called "December Boys." He also made his stage debut in "Equus." Radcliffe anticipates increased media scrutiny now that he is legally an adult but remains committed to staying grounded.	
"""
inputs = tokenizer(
    ARTICLE, return_tensors="pt", truncation=False , padding="max_length", max_length=512
).to(device)

summary_ids = model.generate(
    inputs['input_ids'],
    max_length=300,
    min_length=150,
    length_penalty=2.0,
    num_beams=8,
    early_stopping=False,
    no_repeat_ngram_size=2
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
rouge.get_scores(summary,reference)

In [ ]:
ARTICLE = """
As he awaits a crucial progress report on Iraq, President Bush will try to put a twist on comparisons of the war to Vietnam by invoking the historical lessons of that conflict to argue against pulling out. President Bush pauses Tuesday during a news conference at the North American Leaders summit in Canada. On Wednesday in Kansas City, Missouri, Bush will tell members of the Veterans of Foreign Wars that "then, as now, people argued that the real problem was America's presence and that if we would just withdraw, the killing would end," according to speech excerpts released Tuesday by the White House. "Three decades later, there is a legitimate debate about how we got into the Vietnam War and how we left," Bush will say. "Whatever your position in that debate, one unmistakable legacy of Vietnam is that the price of America's withdrawal was paid by millions of innocent citizens, whose agonies would add to our vocabulary new terms like 'boat people,' 're-education camps' and 'killing fields,' " the president will say. The president will also make the argument that withdrawing from Vietnam emboldened today's terrorists by compromising U.S. credibility, citing a quote from al Qaeda leader Osama bin Laden that the American people would rise against the Iraq war the same way they rose against the war in Vietnam, according to the excerpts. "Here at home, some can argue our withdrawal from Vietnam carried no price to American credibility, but the terrorists see things differently," Bush will say. On Tuesday, Democratic Senate Majority Leader Harry Reid said, "President Bush's attempt to compare the war in Iraq to past military conflicts in East Asia ignores the fundamental difference between the two. Our nation was misled by the Bush Administration in an effort to gain support for the invasion of Iraq under false pretenses, leading to one of the worst foreign policy blunders in our history. "While the President continues to stay-the-course with his failed strategy in Iraq, paid for by the taxpayers, American lives are being lost and there is still no political solution within the Iraqi government. It is time to change direction in Iraq, and Congress will again work to do so in the fall." The White House is billing the speech, along with another address next week to the American Legion, as an effort to "provide broader context" for the debate over the upcoming Iraq progress report by Gen. David Petraeus, the top U.S. military commander, and Ryan Crocker, the U.S. ambassador in Baghdad. President Bush has frequently asked lawmakers -- and the American people -- to withhold judgment on his troop "surge" in Iraq until the report comes out in September. Watch Bush criticize the Iraqi government » . It is being closely watched on Capitol Hill, particularly by Republicans nervous about the political fallout from an increasingly unpopular war. Earlier this month, Defense Secretary Robert Gates said he would wait for the report before deciding when a drawdown of the 160,000 U.S. troops in Iraq might begin. Bush's speeches Wednesday and next week are the latest in a series of attempts by the White House to try to reframe the debate over Iraq, as public support for the war continues to sag. A recent CNN/Opinion Research Corporation poll found that almost two-thirds of Americans -- 64 percent -- now oppose the Iraq war, and 72 percent say that even if Petraeus reports progress, it won't change their opinion. The poll also found a great deal of skepticism about the report; 53 percent said they do not trust Petraeus to give an accurate assessment of the situation in Iraq. In addition to his analogy to Vietnam, Bush in Wednesday's speech will invoke other historical comparisons from Asia, including the U.S. defeat and occupation of Japan after World War II and the Korean War in the 1950s, according to the excerpts. "In the aftermath of Japan's surrender, many thought it naive to help the Japanese transform themselves into a democracy. Then, as now, the critics argued that some people were simply not fit for freedom," Bush will say. "Today, in defiance of the critics, Japan ... stands as one of the world's great free societies." Speaking about the Korean War, Bush will note that at the time "critics argued that the war was futile, that we never should have sent our troops in, or that America's intervention was divisive here at home." "While it is true that the Korean War had its share of challenges, America never broke its word," Bush will say. "Without America's intervention during the war, and our willingness to stick with the South Koreans after the war, millions of South Koreans would now be living under a brutal and repressive regime."
"""
reference = """
President Bush is set to deliver a speech to the Veterans of Foreign Wars in Kansas City, Missouri, where he will draw historical parallels between the Vietnam War and the current conflict in Iraq. Bush will argue that withdrawing from Vietnam led to severe consequences, including the suffering of millions of civilians and the emboldening of terrorists. He will cite the rise of terms like "boat people," "re-education camps," and "killing fields" as direct results of America's withdrawal. Bush will also reference Osama bin Laden's belief that the American people would oppose the Iraq war as they did Vietnam, emphasizing that such a withdrawal would compromise U.S. credibility. In contrast, Democratic Senate Majority Leader Harry Reid criticizes Bush's comparison, arguing that the Iraq war was initiated under false pretenses and represents a significant foreign policy blunder. Reid calls for a change in direction, highlighting the lack of political progress in Iraq and the ongoing loss of American lives. The White House frames Bush's speech, along with another to the American Legion next week, as an effort to provide context for the upcoming Iraq progress report by Gen. David Petraeus and Ambassador Ryan Crocker. Bush has repeatedly asked for patience and a wait-and-see approach until the report is released in September. However, public
"""
inputs = tokenizer(
    ARTICLE, return_tensors="pt", truncation=True , padding="max_length", max_length=512
).to(device)

summary_ids = model.generate(
    inputs['input_ids'],
    max_length=300,
    min_length=100,
    length_penalty=2.0,
    num_beams=8,
    early_stopping=False,
    no_repeat_ngram_size=2
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
rouge.get_scores(summary,reference)

In [ ]:
ARTICLE = """
Drivers who were on the Minneapolis bridge when it collapsed told harrowing tales of survival. "The whole bridge from one side of the Mississippi to the other just completely gave way, fell all the way down," survivor Gary Babineau told CNN. "I probably had a 30-, 35-foot free fall. And there's cars in the water, there's cars on fire. The whole bridge is down." He said his back was injured but he determined he could move around. "I realized there was a school bus right next to me, and me and a couple of other guys went over and started lifting the kids off the bridge. They were yelling, screaming, bleeding. I think there were some broken bones." Watch a driver describe his narrow escape » . At home when he heard about the disaster, Dr. John Hink, an emergency room physician, jumped into his car and rushed to the scene in 15 minutes. He arrived at the south side of the bridge, stood on the riverbank and saw dozens of people lying dazed on an expansive deck. They were in the middle of the Mississippi River, which was churning fast, and he had no way of getting to them. He went to the north side, where there was easier access to people. Ambulances were also having a hard time driving down to the river to get closer to the scene. Working feverishly, volunteers, EMTs and other officials managed to get 55 people into ambulances in less than two hours. Occasionally, a pickup truck with a medic inside would drive to get an injured person and bring him back up even ground, Hink told CNN. The rescue effort was controlled and organized, he said; the opposite of the lightning-quick collapse. "I could see the whole bridge as it was going down, as it was falling," Babineau said. "It just gave a rumble real quick, and it all just gave way, and it just fell completely, all the way to the ground. And there was dust everywhere and it was just like everyone has been saying: It was just like out of the movies." Babineau said the rear of his pickup truck was dangling over the edge of a broken-off section of the bridge. He said several vehicles slid past him into the water. "I stayed in my car for one or two seconds. I saw a couple cars fall," he said. "So I stayed in my car until the cars quit falling for a second, then I got out real quick, ran in front of my truck -- because behind my truck was just a hole -- and I helped a woman off of the bridge with me. "I just wanted off the bridge, and then I ran over to the school bus. I started grabbing kids and handing them down. It was just complete chaos." He said most of the children were crying or screaming. He and other rescuers set them on the ground and told them to run to the river bank, but a few needed to be carried because of their injuries. See rescuers clamber over rubble » . Babineau said he had no rescue training. "I just knew what I had to do at the moment." Melissa Hughes, 32, of Minneapolis, told The Associated Press that she was driving home when the western edge of the bridge collapsed under her. "You know that free-fall feeling? I felt that twice," Hughes said. A pickup landed on top of her car, but she was not hurt. "I had no idea there was a vehicle on my car," she told AP. "It's really very surreal." Babineau told the Minneapolis Star-Tribune: "On the way down, I thought I was dead. I literally thought I was dead. "My truck was completely face down, pointed toward the ground, and my truck got ripped in half. It was folded in half, and I can't believe I'm alive." See and hear eyewitness accounts » . Bernie Toivonen told CNN's "American Morning" that his vehicle was on a part of the bridge that ended up tilted at a 45-degree angle. "I knew the deck was going down, there was no question about it, and I thought I was going to die," he said. After the bridge settled and his car remained upright, "I just put in park, turned the key off and said, 'Oh, I'm alive,' " he said.	

"""
reference = """
On August 1, 2007, the I-35W bridge in Minneapolis, Minnesota, collapsed into the Mississippi River during rush hour, causing multiple vehicles to fall into the water and onto the riverbank. Survivor Gary Babineau described a 30- to 35-foot free fall and the chaotic scene, including cars on fire and a school bus with injured children. Babineau and others helped rescue children from the bus, despite the lack of formal training. Dr. John Hink, an emergency room physician, rushed to the scene and found the rescue effort organized, with volunteers and EMTs working to transport 55 injured people to ambulances within two hours. Melissa Hughes, another survivor, experienced a free-fall sensation as her car was struck by a pickup truck but managed to escape unharmed. Bernie Toivonen's vehicle ended up on a 45-degree angle, but he survived by remaining calm and turning off his car. The collapse was sudden and catastrophic, with witnesses comparing it to a scene from a movie.	
"""
inputs = tokenizer(
    ARTICLE, return_tensors="pt", truncation=True , padding="max_length", max_length=512
).to(device)

summary_ids = model.generate(
    inputs['input_ids'],
    max_length=300,
    min_length=100,
    length_penalty=1.5,
    num_beams=12,
    early_stopping=False,
    no_repeat_ngram_size=3
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
rouge.get_scores(summary,reference)

In [ ]:
ARTICLE = """
The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most severe mental illnesses are incarcerated until they're ready to appear in court. Most often, they face drug charges or charges of assaulting an officer --charges that Judge Steven Leifman says are usually "avoidable felonies." He says the arrests often result from confrontations with police. Mentally ill people often won't do what they're told when police arrive on the scene -- confrontation seems to exacerbate their illness and they become more paranoid, delusional, and less likely to follow directions, according to Leifman. So, they end up on the ninth floor severely mentally disturbed, but not getting any real help because they're in jail. We toured the jail with Leifman. He is well known in Miami as an advocate for justice and the mentally ill. Even though we were not exactly welcomed with open arms by the guards, we were given permission to shoot videotape and tour the floor. Go inside the 'forgotten floor' » . At first, it's hard to determine where the people are. The prisoners are wearing sleeveless robes. Imagine cutting holes for arms and feet in a heavy wool sleeping bag -- that's kind of what they look like. They're designed to keep the mentally ill patients from injuring themselves. That's also why they have no shoes, laces or mattresses. Leifman says about one-third of all people in Miami-Dade county jails are mentally ill. So, he says, the sheer volume is overwhelming the system, and the result is what we see on the ninth floor. Of course, it is a jail, so it's not supposed to be warm and comforting, but the lights glare, the cells are tiny and it's loud. We see two, sometimes three men -- sometimes in the robes, sometimes naked, lying or sitting in their cells. "I am the son of the president. You need to get me out of here!" one man shouts at me. He is absolutely serious, convinced that help is on the way -- if only he could reach the White House. Leifman tells me that these prisoner-patients will often circulate through the system, occasionally stabilizing in a mental hospital, only to return to jail to face their charges. It's brutally unjust, in his mind, and he has become a strong advocate for changing things in Miami. Over a meal later, we talk about how things got this way for mental patients. Leifman says 200 years ago people were considered "lunatics" and they were locked up in jails even if they had no charges against them. They were just considered unfit to be in society. Over the years, he says, there was some public outcry, and the mentally ill were moved out of jails and into hospitals. But Leifman says many of these mental hospitals were so horrible they were shut down. Where did the patients go? Nowhere. The streets. They became, in many cases, the homeless, he says. They never got treatment. Leifman says in 1955 there were more than half a million people in state mental hospitals, and today that number has been reduced 90 percent, and 40,000 to 50,000 people are in mental hospitals. The judge says he's working to change this. Starting in 2008, many inmates who would otherwise have been brought to the "forgotten floor" will instead be sent to a new mental health facility -- the first step on a journey toward long-term treatment, not just punishment. Leifman says it's not the complete answer, but it's a start. Leifman says the best part is that it's a win-win solution. The patients win, the families are relieved, and the state saves money by simply not cycling these prisoners through again and again. And, for Leifman, justice is served.	

"""
reference = """
The ninth floor of the Miami-Dade pretrial detention facility, known as the "forgotten floor," houses inmates with severe mental illnesses who are awaiting court appearances. Judge Steven Leifman, a prominent advocate for the mentally ill, explains that many of these inmates face charges like drug offenses or assaulting an officer, often resulting from confrontations with police. These confrontations can exacerbate their mental conditions, leading to their incarceration. Leifman notes that about one-third of all inmates in Miami-Dade county jails are mentally ill, overwhelming the system and leading to inadequate treatment. During a tour of the ninth floor, the harsh conditions are evident: glaring lights, tiny cells, and a lack of basic comforts. Inmates wear sleeveless robes designed to prevent self-harm and are often seen lying or sitting in their cells, some naked. One inmate, convinced he is the son of the president, shouts for help. Leifman highlights the cyclical nature of their confinement, where inmates stabilize temporarily in mental hospitals before returning to jail. Leifman traces the history of mental health treatment, noting that 200 years ago, the mentally ill were often locked up in jails. Over time, they were moved to mental hospitals, but many of these institutions were closed	
"""
inputs = tokenizer(
    ARTICLE, return_tensors="pt", truncation=True , padding="max_length", max_length=512
).to(device)

summary_ids = model.generate(
    inputs['input_ids'],
    max_length=300,
    min_length=100,
    length_penalty=2.0,
    num_beams=8,
    early_stopping=False,
    no_repeat_ngram_size=2
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
rouge.get_scores(summary,reference)

In [ ]:
ARTICLE = """
A magnitude 4.2 earthquake shook the San Francisco area Friday at 4:42 a.m. PT (7:42 a.m. ET), the U.S. Geological Survey reported. The quake left about 2,000 customers without power, said David Eisenhower, a spokesman for Pacific Gas and Light. Under the USGS classification, a magnitude 4.2 earthquake is considered "light," which it says usually causes minimal damage. "We had quite a spike in calls, mostly calls of inquiry, none of any injury, none of any damage that was reported," said Capt. Al Casciato of the San Francisco police. "It was fairly mild." Watch police describe concerned calls immediately after the quake » . The quake was centered about two miles east-northeast of Oakland, at a depth of 3.6 miles, the USGS said. Oakland is just east of San Francisco, across San Francisco Bay. An Oakland police dispatcher told CNN the quake set off alarms at people's homes. The shaking lasted about 50 seconds, said CNN meteorologist Chad Myers. According to the USGS, magnitude 4.2 quakes are felt indoors and may break dishes and windows and overturn unstable objects. Pendulum clocks may stop.	

"""
reference = """
A magnitude 4.2 earthquake struck the San Francisco area at 4:42 a.m. PT on Friday, affecting about 2,000 customers who lost power, according to Pacific Gas and Light. The U.S. Geological Survey (USGS) classified the quake as "light," typically causing minimal damage. San Francisco police reported no injuries or significant damage, with most calls being inquiries about the event. The earthquake was centered about two miles northeast of Oakland, at a depth of 3.6 miles. Oakland police noted that the quake triggered home alarms, and CNN meteorologist Chad Myers stated that the shaking lasted approximately 50 seconds. The USGS indicates that such quakes are generally felt indoors and can cause minor disruptions like breaking dishes and overturning unstable objects.	
"""
inputs = tokenizer(
    ARTICLE, return_tensors="pt", truncation=True , padding="max_length", max_length=512
).to(device)

summary_ids = model.generate(
    inputs['input_ids'],
    max_length=300,
    min_length=100,
    length_penalty=2.0,
    num_beams=8,
    early_stopping=False,
    no_repeat_ngram_size=2
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
rouge.get_scores(summary,reference)

In [ ]:
ARTICLE = """
Police and FBI agents are investigating the discovery of an empty rocket launcher tube on the front lawn of a Jersey City, New Jersey, home, FBI spokesman Sean Quinn said. Niranjan Desai discovered the 20-year-old AT4 anti-tank rocket launcher tube, a one-time-use device, lying on her lawn Friday morning, police said. The launcher has been turned over to U.S. Army officials at the 754th Ordnance Company, an explosive ordnance disposal unit, at Fort Monmouth, New Jersey, Army officials said. The launcher "is no longer operable and not considered to be a hazard to public safety," police said, adding there was no indication the launcher had been fired recently. Army officials said they could not determine if the launcher had been fired, but indicated they should know once they find out where it came from. The nearest military base, Fort Dix, is more than 70 miles from Jersey City. The Joint Terrorism Task Force division of the FBI and Jersey City police are investigating the origin of the rocket launcher and the circumstance that led to its appearance on residential property. "Al Qaeda doesn't leave a rocket launcher on the lawn of middle-aged ladies," said Paul Cruickshank of New York University Law School's Center on Law and Security. A neighbor, Joe Quinn, said the object lying on Desai's lawn looked military, was brown, had a handle and strap, and "both ends were open, like you could shoot something with it." Quinn also said the device had a picture of a soldier on it and was 3 to 4 feet long. An Army official said the device is basically a shoulder-fired, direct-fire weapon used against ground targets -- a modern-day bazooka -- and it is not wire-guided. According to the Web site Globalsecurity.org, a loaded M136 AT4 anti-tank weapon has a 40-inch-long fiberglass-wrapped tube and weighs just 4 pounds. Its 84 millimeter shaped-charge missile can penetrate 14 inches of armor from a maximum of 985 feet. It is used once and discarded.	

"""
reference = """
Police and FBI agents are investigating the discovery of an empty AT4 anti-tank rocket launcher tube on the front lawn of a home in Jersey City, New Jersey. The 20-year-old launcher, a one-time-use device, was found by Niranjan Desai on Friday morning. The launcher has been turned over to the 754th Ordnance Company at Fort Monmouth for disposal. Authorities stated that the launcher is no longer operable and not a public safety hazard, with no signs of recent use. The nearest military base, Fort Dix, is over 70 miles away, and the origin of the launcher remains under investigation by the Joint Terrorism Task Force and Jersey City police. Experts, including Paul Cruickshank from New York University Law School's Center on Law and Security, have downplayed the likelihood of terrorist involvement, noting that such a device being left on a residential lawn is unusual. The launcher is a shoulder-fired weapon used against ground targets, similar to a modern bazooka, and is typically discarded after use.	
"""
inputs = tokenizer(
    ARTICLE, return_tensors="pt", truncation=True , padding="max_length", max_length=512
).to(device)

summary_ids = model.generate(
    inputs['input_ids'],
    max_length=300,
    min_length=100,
    length_penalty=2.0,
    num_beams=8,
    early_stopping=False,
    no_repeat_ngram_size=2
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
rouge.get_scores(summary,reference)

In [ ]:
ARTICLE = """
A chronology of bombings and attempted bomb attacks in the mainland UK since the 1970s: . Police close off streets around Haymarket, in London's busy theater district. June 29, 2007: Police defuse a bomb consisting of 200 liters of fuel, gas cylinders and nails found in an abandoned car in Haymarket, central London. A second car packed with gas and nails was later found to have been parked just a few hundred yards from the first, before it was towed away by traffic wardens in the early hours of Friday for violating parking restrictions. Police say two vehicles clearly linked. July 21, 2005: Two weeks after the deadly 7/7 bombings, four men are alleged to have attempted to carry out a second wave of attacks against London's transport network at three London underground stations and aboard a bus. But their alleged rucksack bombs fail to explode. July 7, 2005: Four suicide bombers detonate themselves aboard three underground trains and a bus in a morning rush hour attack against London's transport network, killing 52 people and injuring around 700 more. Al Qaeda claims responsibility in a video statement. August 2004: Anti-terrorist police disrupt a plot by Islamic militants to blow up targets including the Ministry of Sound nightclub and the Bluewater shopping center in southeast England using explosives packed into limousines and large vehicles. Seven men are convicted in May 2007 and sentenced to up to 26 years in prison. March 2001: A car bomb explodes outside the BBC's London headquarters, wounding one man. Police blame the Real IRA, a republican splinter group opposed to the IRA's cease fire. April 1999: Three people die when a nail bomb explodes in the Admiral Duncan pub in London's gay district -- the third in a spate of series of nail bomb attacks also targeting immigrant areas of the city that left dozens injured. A 23-year-old self-declared "Nazi", David Copeland, is sentenced to six life terms. June 1996: A massive IRA bomb explodes in a shopping center in central Manchester, injuring more than 200 people. February 1996: Two people die as IRA terrorists detonate a bomb in London's Docklands area, causing damage estimated at around $170m and ending the group's 17-month cease fire. April 1993: An IRA truck bomb devastates part of London's financial district, killing one and wounding 44. March 1993: Two boys aged three and 12 are killed and dozens are injured by two bombs left in litter bins in Warrington, northern England. The IRA admits planting the bombs. April 1992: A huge IRA car bomb in London's financial district kills three people and wounds 91. February 1991: IRA terrorists launch a mortar attack at Prime Minister John Major's Downing Street offices. No-one is injured. September 1989: Eleven people die and 22 are wounded when an IRA bomb explodes at a Royal Marine music school in Deal, southern England. December 1988: A Pan Am airliner explodes over the Scottish town of Lockerbie, killing 259 aboard and 11 people on the ground. Libyan agent Abdel Basset al-Megrahi, convicted of the attack in 2001, was this week granted the right to mount a fresh appeal. (Read about Lockerbie bomber) October 1984: Five people die in an IRA bomb attack on a hotel in Brighton, southern England, where Prime Minister Margaret Thatcher and her cabinet are staying for the Conservative Party's annual conference. December 1983: An IRA bomb at London's Harrods department store kills six people. July 1982: Two IRA bomb attacks on soldiers in London's parks kill 11 people and wound 50. October-November 1974: A wave of IRA bombs in British pubs in Birmingham and Guildford kill 28 people and wound more than 200. February 1974: A coach carrying soldiers and families in northern England is bombed by the IRA, killing 12 and wounding 14.	

"""
reference = """
A chronology of significant bombings and attempted bomb attacks in the UK since the 1970s highlights a series of terrorist activities: - **June 29, 2007**: Police defuse a bomb in an abandoned car in London's Haymarket, and a second car with similar explosives is found nearby. - **July 21, 2005**: Four men attempt to bomb London's transport network but their devices fail to explode, two weeks after the 7/7 attacks. - **July 7, 2005**: Four suicide bombers kill 52 and injure 700 in coordinated attacks on London's transport system; Al Qaeda claims responsibility. - **August 2004**: Anti-terrorist police thwart a plot to bomb the Ministry of Sound and Bluewater shopping center using explosives in limousines. - **March 2001**: A Real IRA car bomb outside the BBC headquarters in London injures one person. - **April 1999**: A nail bomb in London's Admiral Duncan pub kills three and injures dozens; David Copeland, a self-declared Nazi, is convicted. - **June 1996**: An IRA bomb	
"""
inputs = tokenizer(
    ARTICLE, return_tensors="pt", truncation=True , padding="max_length", max_length=512
).to(device)

summary_ids = model.generate(
    inputs['input_ids'],
    max_length=300,
    min_length=100,
    length_penalty=2.0,
    num_beams=8,
    early_stopping=False,
    no_repeat_ngram_size=2
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
rouge.get_scores(summary,reference)